In [3]:
!pip3 install lightgbm

In [4]:
!pip3 install xgboost

In [5]:
!pip3 install catboost

In [36]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 1.1 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.4 MB/s eta 0:00:00


In [206]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from scipy.optimize import minimize
import optuna

In [240]:
train = pd.read_csv("train.csv")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42100 entries, 0 to 42099
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    42100 non-null  int64  
 1   no_of_adults                          42100 non-null  int64  
 2   no_of_children                        42100 non-null  int64  
 3   no_of_weekend_nights                  42100 non-null  int64  
 4   no_of_week_nights                     42100 non-null  int64  
 5   type_of_meal_plan                     42100 non-null  int64  
 6   required_car_parking_space            42100 non-null  int64  
 7   room_type_reserved                    42100 non-null  int64  
 8   lead_time                             42100 non-null  int64  
 9   arrival_year                          42100 non-null  int64  
 10  arrival_month                         42100 non-null  int64  
 11  arrival_date   

In [216]:
# Загрузка данных
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
extra_data = pd.read_csv("train__dataset.csv")

train = pd.concat([train, extra_data], ignore_index=True)

In [218]:
# добавление полиномиальных признаков
numerical_features = ['lead_time', 'avg_price_per_room']
poly = PolynomialFeatures(degree=2, include_bias=False)
train_poly = poly.fit_transform(train[numerical_features])
test_poly = poly.transform(test[numerical_features])

# Предобработка данных
train.drop("id", axis=1, inplace=True)
test.drop("id", axis=1, inplace=True)
train_targets = train["booking_status"]
extra_targets = extra_data["booking_status"]
train.drop("booking_status", axis=1, inplace=True)
extra_data.drop("booking_status", axis=1, inplace=True)

# Обнаружение выбросов
iso_forest = IsolationForest(contamination=0.01, random_state=42)
numerical_cols = train.select_dtypes(include=[np.number]).columns
outliers = iso_forest.fit_predict(train[numerical_cols])
print(len(outliers))
print(sum(outliers))

# Удаление выбросов
train = train[outliers == 1]
train_targets = train_targets[outliers == 1]

# Обнаружение категориальных признаков
cat_cols = [col for col in train.columns if train[col].dtype == 'object' or isinstance(train[col].dtype, pd.StringDtype)]
cat_idxs = [i for i, col in enumerate(train.columns) if train[col].dtype == 'object' or isinstance(train[col].dtype, pd.StringDtype)]

60237
59031


In [220]:
# Начало обучения моделей
test_preds = np.zeros(len(test))
cv = 0
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for fold, (train_idx, valid_idx) in enumerate(kf.split(train, train_targets)):
    print("-"*10 + ">" + f"Fold {fold+1}" + "<" + "-"*10)
    
    X_train, X_valid = train.iloc[train_idx], train.iloc[valid_idx]
    y_train, y_valid = train_targets.iloc[train_idx], train_targets.iloc[valid_idx]
            
    # Сброс индексов
    X_train = X_train.reset_index(drop=True)
    
    # LightGBM
    lgb = LGBMClassifier(
        n_estimators=10000, 
        early_stopping_rounds=100, 
        **{'max_depth': 7, 
           'subsample': 0.5528027451731908, 
           'colsample_bytree': 0.26714040676512085, 
           'learning_rate': 0.11430358849370452}
    )
    lgb.fit(X_train, y_train, eval_set=(X_valid, y_valid))
    
    lgb_preds = lgb.predict_proba(X_valid)[:,1]
    lgb_test_preds = lgb.predict_proba(test)[:,1]
    
    print(roc_auc_score(y_valid, lgb_preds))
    
    # XGBoost
    xgb = XGBClassifier(
        n_estimators=10000, 
        early_stopping_rounds=100,
        **{'max_depth': 12, 
           'subsample': 0.5549531235327668, 
           'colsample_bytree': 0.20335345531973037, 
           'learning_rate': 0.027251718255125106}
    )
    xgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)
    
    xgb_preds = xgb.predict_proba(X_valid)[:,1]
    xgb_test_preds = xgb.predict_proba(test)[:,1]
    
    print(roc_auc_score(y_valid, xgb_preds))
    
    # CatBoost
    ctb = CatBoostClassifier(
        n_estimators=10000, 
        early_stopping_rounds=100, 
        **{'depth': 3, 
           'subsample': 0.9037951675853159, 
           'l2_leaf_reg': 4.085968446512874, 
           'learning_rate': 0.33153188532829714}
    )
    ctb.fit(X_train, y_train, eval_set=(X_valid, y_valid), verbose=0)
    
    ctb_preds = ctb.predict_proba(X_valid)[:,1]
    ctb_test_preds = ctb.predict_proba(test)[:,1]
    
    print(roc_auc_score(y_valid, ctb_preds))
            
    # Мета обучение (stacking/blending)
    meta_train = [lgb_preds, ctb_preds, xgb_preds]
    meta_test = [lgb_test_preds, ctb_test_preds, xgb_test_preds]

    def roc_auc(weights):
        fpred = np.zeros(len(meta_train[0]))
        for i, pred in enumerate(meta_train):
            fpred += weights[i] * pred
        return -roc_auc_score(y_valid, fpred)

    starting_values = [0.33] * len(meta_train) 
    cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
    bounds = [(0,1)] * len(meta_train)  # типичные границы от 0 до 1
    res = minimize(roc_auc, starting_values, method='Nelder-Mead', bounds=bounds, constraints=cons)
    print(-res["fun"])
    
    cv += -res["fun"]
    
    for i, pred in enumerate(meta_test):
        test_preds += res["x"][i] * pred
        
test_preds /= kf.n_splits

---------->Fold 1<----------
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[LightGBM] [Info] Number of positive: 20176, number of negative: 33494
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003229 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 645
[LightGBM] [Info] Number of data points in the train set: 53670, number of used features: 17
[LightGBM] [Warning] early_stopping_round is set=100, early_stopping_rounds=100 will be ignored. Current value: early_stopping_round=100
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.375927 -> initscore=-0.506873
[LightGBM] [Info] Start training from score -0.506873
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

/var/folders/1q/z08d8tcx7xgfz1mb5_zy4__h0000gn/T/ipykernel_74963/3049272178.py:76: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  res = minimize(roc_auc, starting_values, method='Nelder-Mead', bounds=bounds, constraints=cons)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/var/folders/1q/z08d8tcx7xgfz1mb5_zy4__h0000gn/T/ipykernel_74963/3049272178.py:76: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  res = minimize(roc_auc, starting_values, method='Nelder-Mead', bounds=bounds, constraints=cons)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

/var/folders/1q/z08d8tcx7xgfz1mb5_zy4__h0000gn/T/ipykernel_74963/3049272178.py:76: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  res = minimize(roc_auc, starting_values, method='Nelder-Mead', bounds=bounds, constraints=cons)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/var/folders/1q/z08d8tcx7xgfz1mb5_zy4__h0000gn/T/ipykernel_74963/3049272178.py:76: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  res = minimize(roc_auc, starting_values, method='Nelder-Mead', bounds=bounds, constraints=cons)


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/var/folders/1q/z08d8tcx7xgfz1mb5_zy4__h0000gn/T/ipykernel_74963/3049272178.py:76: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  res = minimize(roc_auc, starting_values, method='Nelder-Mead', bounds=bounds, constraints=cons)


In [221]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [222]:
submission = pd.read_csv("sample_submission.csv")

In [223]:
#https://www.kaggle.com/competitions/playground-series-s3e7/discussion/388992

submission["booking_status"] = test_preds / (fold+1)

y = 'booking_status'
dup_features = test.drop(columns='id').columns.tolist()
values_to_assign = test.merge(train.drop(columns='id'), on=dup_features, how='inner')[['id', y]]
map_di = {0: submission[y].max(), 1: submission[y].min()}  #  {0: 1, 1: 0} in a simple way
submission.loc[submission.id.isin(values_to_assign.id), y] = values_to_assign[y].map(map_di).values

In [224]:
submission.to_csv("submission_final.csv", index=False)
submission

,id,booking_status
0,42100,0.010639
1,42101,0.003730
2,42102,0.023175
3,42103,0.003632
4,42104,0.038132
...,...,...
28063,70163,0.102622
28064,70164,0.004055
28065,70165,0.009066
28066,70166,0.054006
